In [1]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm

In [2]:
from pathlib import Path

# Get the current working directory
current_path = Path.cwd()

# Get the parent directory
parent_path = current_path
data_paths = parent_path / 'datas'  

print(data_paths)
#
train_path = data_paths / 'train.csv'
test_path = data_paths / 'test.csv'

train_path_ts = data_paths / 'series_train.parquet'
test_path_ts = data_paths / 'series_test.parquet'

/Users/nayelbenabdesadok/GitProjects/CMI_problematic_internet_use/datas


In [3]:
# Load datasets

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

## Pre processing

In [4]:
features_to_keep = ['id', 'Basic_Demos-Age', 'Basic_Demos-Sex','CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP','Physical-HeartRate','Physical-Systolic_BP','Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Sec', 
                    'FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone',
                    'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                    'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num','BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW',
                    'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
                    'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14','PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18','PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
                    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday',
                    'sii']


# Select the specified columns from the train DataFrame
train_selected = train[features_to_keep].copy()

In [5]:
columns_to_drop = []

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = train[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")
    
    if ratio > 0.7:
        columns_to_drop.append(column)

# Drop the columns with NaN ratio greater than 0.7
train_selected.drop(columns=columns_to_drop, inplace=True)

print(f"Columns dropped: {columns_to_drop}")

The ratio for id is 0.0
The ratio for Basic_Demos-Age is 0.0
The ratio for Basic_Demos-Sex is 0.0
The ratio for CGAS-CGAS_Score is 0.3886363636363636
The ratio for Physical-BMI is 0.23686868686868687
The ratio for Physical-Diastolic_BP is 0.25404040404040407
The ratio for Physical-HeartRate is 0.25075757575757573
The ratio for Physical-Systolic_BP is 0.25404040404040407
The ratio for Fitness_Endurance-Max_Stage is 0.8123737373737374
The ratio for Fitness_Endurance-Time_Sec is 0.8131313131313131
The ratio for FGC-FGC_CU_Zone is 0.42373737373737375
The ratio for FGC-FGC_GSND_Zone is 0.7318181818181818
The ratio for FGC-FGC_GSD_Zone is 0.7315656565656565
The ratio for FGC-FGC_PU_Zone is 0.4265151515151515
The ratio for FGC-FGC_SRL_Zone is 0.42752525252525253
The ratio for FGC-FGC_SRR_Zone is 0.427020202020202
The ratio for FGC-FGC_TL_Zone is 0.42297979797979796
The ratio for BIA-BIA_Activity_Level_num is 0.49722222222222223
The ratio for BIA-BIA_BMC is 0.49722222222222223
The ratio for BI

## Strategy

We can make category per Age/sex

for float take median 

In [6]:
# Identify float type columns
float_columns = train_selected.select_dtypes(include=['float64']).columns

print(float_columns)

# Group by 'Basic_Demos-Age' and 'Basic_Demos-Sex'
grouped = train_selected.groupby(['Basic_Demos-Age', 'Basic_Demos-Sex'])

# Function to fill NaN values with the median of the group
def fill_na_with_median(group):
    for column in float_columns:
        median_value = group[column].median()
        group[column].fillna(median_value, inplace=True)
    return group

# Apply the function to each group
train_selected = grouped.apply(fill_na_with_median)


print(train_selected)

Index(['CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP', 'FGC-FGC_CU_Zone',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone',
       'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC',
       'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW',
       'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat',
       'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')


/var/folders/sg/lshv0q7145ggm8z9kc3b1y4r0000gn/T/ipykernel_66540/3596357927.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  group[column].fillna(median_value, inplace=True)


                                            id  Basic_Demos-Age  \
Basic_Demos-Age Basic_Demos-Sex                                   
5               0               0     00008ff9                5   
                                210   0bd89878                5   
                                267   0f6b71c4                5   
                                297   1184a14a                5   
                                344   142242d9                5   
...                                        ...              ...   
22              1               1522  5f284ef1               22   
                                1602  653f46e1               22   
                                2048  81e9a24a               22   
                                2345  940b2f17               22   
                                3297  d50a8c3c               22   

                                      Basic_Demos-Sex  CGAS-CGAS_Score  \
Basic_Demos-Age Basic_Demos-Sex                       

/var/folders/sg/lshv0q7145ggm8z9kc3b1y4r0000gn/T/ipykernel_66540/3596357927.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_selected = grouped.apply(fill_na_with_median)


In [8]:
from sklearn.impute import KNNImputer

# Drop ID columns or any non-relevant columns

ids = train_selected['id']
train_selected_without_ids = train_selected.drop(['id'], axis=1)

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=20)

# Fit and transform the data
imputed_data = imputer.fit_transform(train_selected_without_ids)

# Optionally, convert the result back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=train_selected_without_ids.columns)

imputed_df['id'] = ids.values

In [9]:
features_to_keep = imputed_df.columns

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = imputed_df[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")

The ratio for Basic_Demos-Age is 0.0
The ratio for Basic_Demos-Sex is 0.0
The ratio for CGAS-CGAS_Score is 0.0
The ratio for Physical-BMI is 0.0
The ratio for Physical-Diastolic_BP is 0.0
The ratio for Physical-HeartRate is 0.0
The ratio for Physical-Systolic_BP is 0.0
The ratio for FGC-FGC_CU_Zone is 0.0
The ratio for FGC-FGC_PU_Zone is 0.0
The ratio for FGC-FGC_SRL_Zone is 0.0
The ratio for FGC-FGC_SRR_Zone is 0.0
The ratio for FGC-FGC_TL_Zone is 0.0
The ratio for BIA-BIA_Activity_Level_num is 0.0
The ratio for BIA-BIA_BMC is 0.0
The ratio for BIA-BIA_BMI is 0.0
The ratio for BIA-BIA_BMR is 0.0
The ratio for BIA-BIA_DEE is 0.0
The ratio for BIA-BIA_ECW is 0.0
The ratio for BIA-BIA_FFM is 0.0
The ratio for BIA-BIA_FFMI is 0.0
The ratio for BIA-BIA_FMI is 0.0
The ratio for BIA-BIA_Fat is 0.0
The ratio for BIA-BIA_Frame_num is 0.0
The ratio for BIA-BIA_ICW is 0.0
The ratio for BIA-BIA_LDM is 0.0
The ratio for BIA-BIA_LST is 0.0
The ratio for BIA-BIA_SMM is 0.0
The ratio for BIA-BIA_TBW 